In [10]:
import cv2
import numpy as np
from typing import Tuple, List

In [11]:
!pip install ultralytics

###The Function below uses a provided distance map to apply a spatially varying Gaussian blur to an image

For our implementation purposes, the distance map specifically assumes increasing distance from bottom of the image to the top of the image

In [12]:
def spatially_varying_gaussian_blur(image: np.ndarray, distance_map: np.ndarray, blur_start: float, blur_growth_rate: float, num_levels: int = 5) -> np.ndarray:

    # Make sure the distance_map has the same shape as image
    if distance_map.shape != image.shape[:2]:
        distance_map = cv2.resize(distance_map, (image.shape[1], image.shape[0]))

    # 1. Calculate Continuous Sigma Map
    sigma_map = np.zeros_like(distance_map, dtype=float)
    mask = distance_map > blur_start
    sigma_map[mask] = (distance_map[mask] - blur_start) * blur_growth_rate
    sigma_map = np.maximum(0, sigma_map)  # Make sure the sigma is non-negative

    # 2. Define Discrete Levels
    max_sigma = np.max(sigma_map)
    if max_sigma == 0:
        return image.copy()

    sigma_levels = np.linspace(0, max_sigma, num_levels)
    # Make sure sigma_levels[0] is exactly 0
    sigma_levels[0] = 0

    # Calculate corresponding kernel sizes (odd integers)
    kernel_sizes = [
        max(1, int(round(6 * sigma)) * 2 + 1) if sigma > 0 else 1
        for sigma in sigma_levels
    ]

    # 3. Pre-compute Blurred Images
    blurred_images = []
    for i in range(num_levels):
        if sigma_levels[i] > 0:
            ksize = kernel_sizes[i]
            # Make sure the kernel size is odd
            if ksize % 2 == 0:
                ksize += 1
            blurred_images.append(
                cv2.GaussianBlur(image, (ksize, ksize), sigma_levels[i])
            )
        else:
            blurred_images.append(image.copy())  # If sigma is 0, use the original image

    # 4. Create Masks and Composite
    composite_image = np.zeros_like(image, dtype=float)

    # lowest blur level (level 0)
    mask = sigma_map <= (sigma_levels[1] / 2) if num_levels > 1 else np.ones_like(sigma_map, dtype=bool)
    mask = mask[..., np.newaxis]  # Add channel dimension
    composite_image = np.where(mask, blurred_images[0], composite_image)

    # intermediate blur level
    for i in range(1, num_levels - 1):
        lower_threshold = (sigma_levels[i-1] + sigma_levels[i]) / 2
        upper_threshold = (sigma_levels[i] + sigma_levels[i+1]) / 2
        mask = (sigma_map > lower_threshold) & (sigma_map <= upper_threshold)
        mask = mask[..., np.newaxis]  # Add channel dimension
        composite_image = np.where(mask, blurred_images[i], composite_image)

    # highest blur level
    if num_levels > 1:
        mask = sigma_map > ((sigma_levels[-2] + sigma_levels[-1]) / 2)
        mask = mask[..., np.newaxis]  # Add channel dimension
        composite_image = np.where(mask, blurred_images[-1], composite_image)

    return composite_image.astype(np.uint8)

### The function below uses a distance_map to apply a spacially varying Gaussian blur to a given image using two-passes.

The two-pass blurring technique is a technique where the background and foreground are blurred seperately then combined.

Two-pass was chosen because it is faster and more efficient than a full 2D convolution. Efficiency is important because our dataset we process is 7000 images

In [13]:
def two_pass_blur(image: np.ndarray, segmentation_mask: np.ndarray, distance_map: np.ndarray, blur_start: float, blur_growth_rate: float, num_levels: int = 5) -> np.ndarray:

    # Make sure the segmentation_mask has the correct shape and is a boolean array
    segmentation_mask = cv2.resize(segmentation_mask, (image.shape[1], image.shape[0]))
    segmentation_mask = segmentation_mask.astype(bool)

    # Ensure distance_map matches the image shape
    if distance_map.shape != image.shape[:2]:
        distance_map = cv2.resize(distance_map, (image.shape[1], image.shape[0]))

    # 1. Background Blurring
    # Create a mask to prevent blurring the foreground
    bg_mask = ~segmentation_mask  # Invert the mask
    bg_distance_map = np.where(bg_mask, distance_map, 0)  # Only blur the background
    blurred_background = spatially_varying_gaussian_blur(
        image, bg_distance_map, blur_start, blur_growth_rate, num_levels
    )

    # 2. Foreground Blurring
    # Create a mask to isolate the foreground
    fg_mask = segmentation_mask
    fg_distance_map = np.where(fg_mask, distance_map, 0)  # Only blur the foreground
    blurred_foreground = spatially_varying_gaussian_blur(
        image, fg_distance_map, blur_start, blur_growth_rate, num_levels
    )

    # 3. Final Composition
    # Add channel dimension to mask for broadcasting if needed
    if len(segmentation_mask.shape) == 2 and len(image.shape) == 3:
        mask_3d = segmentation_mask[..., np.newaxis]
    else:
        mask_3d = segmentation_mask

    # Combine the blurred background and foreground
    final_image = np.where(mask_3d, blurred_foreground, blurred_background)
    return final_image.astype(np.uint8)

### The function below uses Ultralytics YOLO we imported earlier to segment the foreground object in the image

In [14]:
def segment_with_yolo(image: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:

    try:
        from ultralytics import YOLO
    except ImportError:
        raise ImportError(
            "Ultralytics YOLO is required. Install it with: pip install ultralytics"
        )

    # Load pre-trained YOLOv8 segmentation model
    model = YOLO('yolov8s-seg.pt')

    # Run inference on the image
    results = model.predict(image)

    # Check if any objects were detected
    if len(results) == 0 or results[0].masks is None:
        print("No objects detected by YOLO.")
        return image, np.zeros(image.shape[:2], dtype=np.uint8)  # Return an empty mask

    # Extract the mask for the first detected object
    mask = results[0].masks.data[0].cpu().numpy()

    # Convert mask to 0s and 1s
    mask = (mask > 0.5).astype(np.uint8)

    # Resize mask to match image dimensions if necessary
    if mask.shape != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]))

    # Return original image and mask
    return image, mask

### The function below creates a distance_map.

For our implementation, we made the distance map increase from the bottom to the top of the image because the objects at the bottom of the image are typically closer to the camera lens and the objects at the top.

In [15]:
def create_bottom_to_top_distance_map(image_shape: Tuple[int, int], min_distance: float, max_distance: float) -> np.ndarray:

    height, width = image_shape
    # Create distance values from top to bottom of image
    distance_map = np.linspace(max_distance, min_distance, height)
    # Expand to 2D
    distance_map = np.tile(distance_map[:, np.newaxis], (1, width))
    return distance_map

### Below is the main method that executes the above functions in correct order with proper arguements

In [16]:
def main():

    # 1. Load an image
    image_path = "/content/drive/MyDrive/testerimage.jpg"  # Imagepath, will be replaced every time we process a new image from the dataset
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image at {image_path}")
        return

    # Convert BGR to RGB for YOLO
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 2. Segment the image using YOLO
    _, segmentation_mask = segment_with_yolo(image_rgb)

    # Check if the segmentation mask is empty
    if np.sum(segmentation_mask) == 0:
        print("No valid segmentation mask obtained. Exiting.")
        return

    # 3. Create a distance map that increases from bottom to top
    height, width = image.shape[:2]
    distance_map = create_bottom_to_top_distance_map(
        image_shape=(height, width),
        min_distance=10,  # Distance value at the bottom
        max_distance=100  # Distance value at the top
    )

    # 4. Set blurring parameters
    blur_start = 30.0
    blur_growth_rate = 0.1
    num_levels = 5  # Number of discrete blur levels

    # 5. Apply the two-pass blur
    final_image = two_pass_blur(image, segmentation_mask, distance_map, blur_start, blur_growth_rate, num_levels)

    # 6. Save the results
    cv2.imwrite("original_image.jpg", image)
    cv2.imwrite("segmentation_mask.jpg", segmentation_mask * 255)

    # Visualize the distance map
    normalized_distance_map = cv2.normalize(distance_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    distance_map_colored = cv2.applyColorMap(normalized_distance_map, cv2.COLORMAP_JET)
    cv2.imwrite("distance_map.jpg", distance_map_colored)

    cv2.imwrite("final_blurred_image.jpg", final_image)

    # 7. Display the results
    try:
        cv2.imshow("Original Image", image)
        cv2.imshow("Segmentation Mask", segmentation_mask * 255)
        cv2.imshow("Distance Map", distance_map_colored)
        cv2.imshow("Final Blurred Image", final_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    except:
        print("Images saved but not displayed")

if __name__ == "__main__":
    main()

Error: Could not read image at /content/drive/MyDrive/testerimage.jpg
